## Part 1

What is the value of the recovered frequency (the value of the most recently played sound) the first time a `rcv` instruction is executed with a non-zero value?

In [1]:
def run(instructions):
    SND_REG = 0
    regs = {SND_REG: 0} # Registers. 0 is the special register that keeps the result of `snd X`

    def get_value(y): # Returns y when y is an int and the value of register named y otherwise
        try:
            return int(y)
        except ValueError:
            if not y in regs:
                regs[y] = 0
            return regs[y]

    i = 0
    while i < len(instructions):
        spl = instructions[i].split(' ')
        cmd = spl[0]
        x = spl[1]
        if cmd == 'snd':
            regs[SND_REG] = get_value(x)
        elif cmd == 'set':        
            regs[x] = get_value(spl[2])
        elif cmd == 'add':
            regs[x] = get_value(x) + get_value(spl[2])
        elif cmd == 'mul':
            regs[x] = get_value(x) * get_value(spl[2])
        elif cmd == 'mod':
            regs[x] = get_value(x) % get_value(spl[2])
        elif cmd == 'rcv':
            if get_value(x) != 0 and regs[SND_REG] != 0:
                return regs[SND_REG], regs
        elif cmd == 'jgz':
            if get_value(x) > 0:
                i += get_value(spl[2])
                continue
        i += 1

    return regs

In [2]:
test_in = 'set a 1,add a 2,mul a a,mod a 5,snd a,set a 0,rcv a,jgz a -1,set a 1,\
jgz a -2'.split(',')
expected = 4
actual, registers = run(test_in)
assert expected == actual, 'run(test_in) failed, expected {}, got {} {}'.format(
    expected, actual, registers)

In [3]:
puzzle_in = [l[:-1] if l.endswith('\n') else l for l in open('day18.txt', 'r')]
expected = 2951
actual, registers = run(puzzle_in)
assert expected == actual, 'run(puzzle_in) failed, expected {}, got {} {}'.format(
    expected, actual, registers)
print('Part 1 answer: {} {}'.format(actual, registers))

Part 1 answer: 2951 {0: 2951, 'i': 126, 'a': 2147483647, 'p': 1842102951, 'b': 2951, 'f': 0}


## Part 2 - Two threads

In [4]:
def get_value(proc, y):
    try:
        return int(y)
    except ValueError:
        if not y in proc:
            proc[y] = 0
        return proc[y]

# Special register names
ID_REG, Q_REG, OUT_CNT, POS_REG = 'p', 'que', 'out_cnt', 'pos'

def run_duet(instructions):
    pr0 = {ID_REG: 0, Q_REG: [], OUT_CNT: 0, POS_REG: 0} # Processes whose ID = 0
    pr1 = {ID_REG: 1, Q_REG: [], OUT_CNT: 0, POS_REG: 0} # Processes whose ID = 1
    while True:
        #print(pr0, pr1)
        s0, s1 = do_step(pr0, pr1[Q_REG], instructions), do_step(pr1, pr0[Q_REG], instructions)
        if s0 and s1:
            break
    return pr0, pr1

def do_step(proc, out, instructions):
    if proc[POS_REG] > len(instructions) - 1:
        return True  # Process finished
    spl = instructions[proc[POS_REG]].split(' ')
    cmd = spl[0]
    x = spl[1]
    if cmd == 'snd':
        out.insert(0, get_value(proc, x))
        proc[OUT_CNT] += 1
    elif cmd == 'set':        
        proc[x] = get_value(proc, spl[2])
    elif cmd == 'add':
        proc[x] = get_value(proc, x) + get_value(proc, spl[2])
    elif cmd == 'mul':
        proc[x] = get_value(proc, x) * get_value(proc, spl[2])
    elif cmd == 'mod':
        proc[x] = get_value(proc, x) % get_value(proc, spl[2])
    elif cmd == 'rcv':
        if len(proc[Q_REG]) == 0:  # Incoming queue is empty, waiting
            return True
        proc[x] = proc[Q_REG].pop()
    elif cmd == 'jgz':
        if get_value(proc, x) > 0:
            proc[POS_REG] += get_value(proc, spl[2])
            return False
    proc[POS_REG] += 1
    return False

In [5]:
test_instructions = 'snd 1,snd 2,snd p,rcv a,rcv b,rcv c,rcv d'.split(',')
run_duet(test_instructions)

({'a': 1, 'b': 2, 'c': 1, 'out_cnt': 3, 'p': 0, 'pos': 6, 'que': []},
 {'a': 1, 'b': 2, 'c': 0, 'out_cnt': 3, 'p': 1, 'pos': 6, 'que': []})

In [6]:
p0, p1 = run_duet(puzzle_in)
print('Part 2 answer: {}'.format(p1[OUT_CNT]))

Part 2 answer: 7366
